# 📌 Estrutura do Notebook 02B-Ingestao-Bronze-Classificacao
📍 Objetivo: Ler os dados da camada Bronze, aplicar transformações e salvar na Silver.

In [0]:
%run "./00-Configuracao"

Leitura das credenciais de forma segura pelo Spark 🔐


## 🔶 Leitura dos dados da camada Bronze 

In [0]:
# Importar bibliotecas necessárias
from pyspark.sql import SparkSession

# Definir o caminho do arquivo CSV na Bronze
csv_file_path = f"s3a://{aws_bucket_name}/bronze/BrasilSerieA_2024_ClassificacaoFinal.csv"

# Ler o arquivo CSV da camada Bronze
df_bronze = spark.read.csv(csv_file_path, header=True, inferSchema=True, sep=";", encoding='utf-8')

print(f'✅ Leitura do Arquivo BrasilSerieA_2024_ClassificacaoFinal na camada Bronze feita com sucesso ')

✅ Leitura do Arquivo BrasilSerieA_2024_ClassificacaoFinal na camada Bronze feita com sucesso 


✅ Credenciais configuradas com sucesso!


✅ Conexão bem-sucedida! Arquivos encontrados na camada Bronze e Silver:


## 🔶 Aplicar transformações (limpeza e padronização) 

Limpezas que serão aplicadas no arquivo `BrasilSerieA_2024_ClassificacaoFinal.csv`



#### 🔹Renomeando as colunas para deixar mais compreensivel do que se trata cada uma delas

In [0]:
df_bronze = df_bronze.withColumnRenamed("Posi��o", "Posicao") \
                     .withColumnRenamed("Pts", "Pontuacao") \
                     .withColumnRenamed("PJ", "Partidas_Jogadas") \
                     .withColumnRenamed("VIT", "Vitorias") \
                     .withColumnRenamed("E", "Empates") \
                     .withColumnRenamed("DER", "Derrotas") \
                     .withColumnRenamed("GM", "Gols_Marcados") \
                     .withColumnRenamed("GC", "Gols_Sofridos") \
                     .withColumnRenamed("SG", "Saldo_de_Gols")                                    

#### 🔹Ajustando valores dos Clubes, para manter um padrão

In [0]:
# Dicionário de padronização dos nomes dos clubes (em ordem alfabética)
mapeamento_clubes = {
    "Atletico GO": "Atlético-GO",
    "Atletico-MG": "Atlético-MG",
    "Athletico-PR": "Athletico-PR",
    "Bahia": "Bahia",
    "Botafogo RJ": "Botafogo",
    "Bragantino": "Bragantino",
    "Corinthians": "Corinthians",
    "Criciuma": "Criciúma",
    "Cruzeiro": "Cruzeiro",
    "Cuiaba": "Cuiabá",
    "Flamengo RJ": "Flamengo",
    "Fluminense": "Fluminense",
    "Fortaleza": "Fortaleza",
    "Gremio": "Grêmio",
    "Internacional": "Internacional",
    "Juventude": "Juventude",
    "Palmeiras": "Palmeiras",
    "Sao Paulo": "São Paulo",
    "Vasco": "Vasco da Gama",
    "Vitoria": "EC Vitória"
}

df_bronze = df_bronze.replace(mapeamento_clubes, subset=["Clube"])

## 🔶 Analise Geral do arquivo Tratado (Checagem) 


#### 🔹contagem de valores vazios por coluna

In [0]:
from pyspark.sql.functions import col, lit, concat_ws, monotonically_increasing_id, quarter, when, to_date, sum

# Criar um DataFrame com a contagem de valores vazios por coluna
empty_counts = df_bronze.select([sum(col(c).isNull().cast("int")).alias(c) for c in df_bronze.columns])


#### 🔹resumo estatístico

In [0]:
# Exibir resumo estatístico das colunas + contagem de valores vazios
print("📌 Estatísticas do DataFrame:")
df_bronze.summary("count", "min", "max").show()

print("📌 Contagem de valores vazios por coluna:")
empty_counts.show()

print("📌 Verificação do Tipo de Variável de Cada Coluna:")
df_bronze.printSchema()


📌 Estatísticas do DataFrame:
+-------+-------+-------------+---------+----------------+--------+-------+--------+-------------+-------------+-------------+
|summary|Posicao|        Clube|Pontuacao|Partidas_Jogadas|Vitorias|Empates|Derrotas|Gols_Marcados|Gols_Sofridos|Saldo_de_Gols|
+-------+-------+-------------+---------+----------------+--------+-------+--------+-------------+-------------+-------------+
|  count|     20|           20|       20|              20|      20|     20|      20|           20|           20|           20|
|    min|      1| Athletico-PR|       30|              38|       6|      7|       5|           29|           29|          -29|
|    max|     20|Vasco da Gama|       79|              38|      23|     14|      22|           61|           61|           30|
+-------+-------+-------------+---------+----------------+--------+-------+--------+-------------+-------------+-------------+

📌 Contagem de valores vazios por coluna:
+-------+-----+---------+---------------

## 💾 Salvar os dados na camada Silver (Parquet no S3)

#### 🔹 Salvando o arquivo no S3 camada Silver

In [0]:
# Caminho correto no S3 para armazenar os arquivos
silver_parquet_path = f"s3a://{aws_bucket_name}/silver/parquet/brasil_seriea_2024_classificacao"
silver_delta_path = f"s3a://{aws_bucket_name}/silver/delta/brasil_seriea_2024_classificacao"

# Salvar em Parquet (para acessibilidade geral)
df_bronze.write.mode("overwrite").parquet(silver_parquet_path)

# Salvar em Delta (para controle e governança)
df_bronze.write.format("delta").mode("overwrite").save(silver_delta_path)

print("✅ Dados salvos corretamente em Parquet e Delta, em pastas separadas!")

✅ Dados salvos corretamente em Parquet e Delta, em pastas separadas!


#### 🔹 Salvando o arquivo no DBFS

In [0]:
# Caminho no DBFS para armazenar os dados Silver temporariamente
dbfs_path_silver_classificacao = "dbfs:/mnt/silver_temp/classificacao"

# Salvar no DBFS em formato Parquet
df_bronze.write.mode("overwrite").parquet(dbfs_path_silver_classificacao)

print("✅ Dados transformados e salvos na camada Silver no DBFS com sucesso!")

✅ Dados transformados e salvos na camada Silver no DBFS com sucesso!
